In [3]:
# helps fix a bug between pymc3 and numpy
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as at
import statsmodels.api as sm

In [4]:
# --- __init__.py for forecast_engine package ---

# OLS Engine
from .ols_engine import (
    fit_ols_matrix,
    predict_ols_matrix,
    fit_ols_statsmodels,
    predict_ols_statsmodels,
    fit_ols_sklearn,
    predict_ols_sklearn,
    evaluate_model,
    add_fitted_and_residuals
)

# Bayesian Engine
from .bayesian_engine import (
    fit_bayesian_regression,
    simulate_bayesian_forecasts,
    summarize_bayesian_distribution
)

# Residual Bootstrap Engine
from .bootstrap_engine import (
    simulate_bootstrap_forecasts,
    check_residual_stationarity,
    summarize_bootstrap_distribution
)

# Plotting
from .plotting import (
    plot_true_vs_predicted,
    plot_actual_vs_fitted_vs_forecast,
    plot_selected_forecasts,
    plot_bootstrap_forecast,
    plot_input_variables,
    plot_aggregate_forecast_distribution  # ✅ new
)

# Utilities
from .utils import (
    get_evaluation_metrics,
    summarize_forecast_table,
    summarize_forecast_table_with_colors,
    add_comb_and_flag,
    calculate_yoy,
    convert_weekly_to_monthly,
    convert_weekly_to_fiscal
)

# Preprocessing (if you created forecast_preprocessing.py)
from .forecast_preprocessing import (
    filter_by_date_range,
    check_missing_values,
    drop_or_impute_missing,
    detect_extreme_outliers,
    remove_outliers,
    log_transform,
    winsorize_data,
    scale_features,
    create_lagged_features
)


ImportError: attempted relative import with no known parent package

In [5]:
# --- Step 1: Load and prepare your data ---

# Example data load
# Create the foundational dataset
df = pd.read_csv('grocery_eda_dataset.csv')
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.set_index('date')

# define the variables
dep = 'grocery_sales'
ind = ['cpi_fah', 'rdi_adj', 'home_price', 'covid1', 'covid2']

In [ ]:
# --- Step 2: Define train and test periods ---

# Define Training and Testing periods
start_training_date = '2004-01-01'    # Start of your dataset
end_training_date = '2023-12-31'      # Last month of training data

start_test_date = '2024-01-01'         # First month of testing data
end_test_date = '2025-03-31'           # Last month of testing data

X_train = df.loc[start_training_date:end_training_date, ind]
y_train = df.loc[start_training_date:end_training_date, dep]

X_test = df.loc[start_test_date:end_test_date, ind]
y_test = df.loc[start_test_date:end_test_date, dep]

X_train_fcst = df.loc[start_training_date:end_test_date, ind]
y_train_fcst = df.loc[start_training_date:end_test_date, dep]

df = df[df.index<=end_test_date]

In [ ]:
# --- Step 3: Train OLS Models ----------------------------------------------------

# ✅ Matrix-Based OLS
beta_matrix = fit_ols_matrix(X_train, y_train)
y_fitted_matrix = predict_ols_matrix(X_train, beta_matrix)
evaluate_model(y_train, y_fitted_matrix, model_name="Matrix-Based OLS")
plot_true_vs_predicted(y_train, y_fitted_matrix, title="Matrix-Based OLS")

# ✅ Statsmodels OLS
model_stats, y_fitted_stats = fit_ols_statsmodels(X_train, y_train)
evaluate_model(y_train, y_fitted_stats, model_name="Statsmodels OLS")
plot_true_vs_predicted(y_train, y_fitted_stats, title="Statsmodels OLS")

# ✅ Scikit-Learn OLS
model_sklearn, y_fitted_sklearn = fit_ols_sklearn(X_train, y_train)
evaluate_model(y_train, y_fitted_sklearn, model_name="Scikit-Learn OLS")
plot_true_vs_predicted(y_train, y_fitted_sklearn, title="Scikit-Learn OLS")


In [ ]:
# --- Step 3B: Evaluate All OLS Models on Test Set -----------------------------------------

# ✅ Matrix-Based OLS
y_pred_matrix = predict_ols_matrix(X_test, beta_matrix)
evaluate_model(y_test, y_pred_matrix, model_name="Matrix-Based OLS (Test)")
plot_true_vs_predicted(y_test, y_pred_matrix, title="Matrix-Based OLS (Test)")

# ✅ Statsmodels OLS
y_pred_stats = model_stats.predict(sm.add_constant(X_test))
evaluate_model(y_test, y_pred_stats, model_name="Statsmodels OLS (Test)")
plot_true_vs_predicted(y_test, y_pred_stats, title="Statsmodels OLS (Test)")

# ✅ Scikit-Learn OLS
y_pred_sklearn = model_sklearn.predict(X_test)
evaluate_model(y_test, y_pred_sklearn, model_name="Scikit-Learn OLS (Test)")
plot_true_vs_predicted(y_test, y_pred_sklearn, title="Scikit-Learn OLS (Test)")


In [ ]:
# --- Step 4: Retrain on Full 2004–2024 for Forecasting ---

# ✅ Matrix-Based OLS
beta_fcst_matrix = fit_ols_matrix(X_train_fcst, y_train_fcst)
y_fitted_matrix_fcst = predict_ols_matrix(X_train_fcst, beta_fcst_matrix)

# ✅ Statsmodels OLS
model_stats_fcst, y_fitted_stats_fcst = fit_ols_statsmodels(X_train_fcst, y_train_fcst)

# ✅ Scikit-Learn OLS
model_sklearn_fcst, y_fitted_sklearn_fcst = fit_ols_sklearn(X_train_fcst, y_train_fcst)

# ✅ Use Matrix-based fitted values for diagnostics + residuals
df = add_fitted_and_residuals(df, y_train_fcst, y_fitted_matrix_fcst)



In [ ]:
# --- Step 5: Evaluate OLS Models ---

# ✅ Matrix-Based OLS
y_fitted_matrix_fcst = predict_ols_matrix(X_train_fcst, beta_fcst_matrix)
evaluate_model(y_train_fcst, y_fitted_matrix_fcst, model_name="Matrix-Based OLS")

# ✅ Statsmodels OLS
y_fitted_stats_fcst = predict_ols_statsmodels(X_train_fcst, model_stats_fcst)
evaluate_model(y_train_fcst, y_fitted_stats_fcst, model_name="Statsmodels OLS")

# ✅ Scikit-Learn OLS
y_fitted_sklearn_fcst = predict_ols_sklearn(X_train_fcst, model_sklearn_fcst)
evaluate_model(y_train_fcst, y_fitted_sklearn_fcst, model_name="Scikit-Learn OLS")

# 📌 Use matrix-based fitted values for residual diagnostics
df = add_fitted_and_residuals(df, y_train_fcst, y_fitted_matrix_fcst)

In [ ]:
# --- Step 6: Recursive Forecast with YOY Input Conversion -------------------------------------

from grocery_sales_input_202504 import forward_inputs

# 1. Define forecast variables
forecast_vars = ['cpi_fah', 'rdi_adj', 'home_price']
all_vars = forecast_vars + ['covid1', 'covid2']
dep_lag = 'grocery_sales_lag1'

# 2. Extract and sort future periods
future_periods = pd.to_datetime(list(forward_inputs[forecast_vars[0]].keys()))
future_periods = future_periods.sort_values()

# 3. Build forward input levels
X_future_dict = {}
future_array_rows = []

for period in future_periods:
    X_future_dict[period] = {}
    
    for var in forecast_vars:
        ly_period = period - pd.DateOffset(years=1)
        if ly_period not in df.index:
            raise ValueError(f"Missing last-year value for {var} in {ly_period}")
        ly_level = df.loc[ly_period, var]
        yoy_change = forward_inputs[var][period]
        X_future_dict[period][var] = ly_level * (1 + yoy_change / 100.0)

    X_future_dict[period]['covid1'] = 0
    X_future_dict[period]['covid2'] = 0

    row = [X_future_dict[period][v] for v in all_vars]
    future_array_rows.append(row)

# 4. Create input DataFrame
X_forward_df = pd.DataFrame(future_array_rows, columns=all_vars, index=future_periods)
X_forward_df[dep_lag] = np.nan

# 5. Initialize last known sales
last_sales_matrix = df[dep].iloc[-1]
last_sales_stats = last_sales_matrix
last_sales_sklearn = last_sales_matrix

# 6. Containers for forecasts
forecast_ols_matrix = []
forecast_ols_stats = []
forecast_ols_sklearn = []
forecast_ols_statsmodels = []

last_sales_matrix = df[dep].iloc[-1]
last_sales_stats = df[dep].iloc[-1]
last_sales_sklearn = df[dep].iloc[-1]


# 7. Recursive forecasting loop

for i in range(len(X_forward_df)):
    row = X_forward_df.iloc[i].copy()

    # Update lagged sales
    row[dep_lag] = last_sales_matrix
    row_input_matrix = np.array([1] + list(row[ind].values))  # Manual intercept
    y_hat_matrix = np.dot(row_input_matrix, beta_fcst_matrix)
    forecast_ols_matrix.append(y_hat_matrix)
    last_sales_matrix = y_hat_matrix

    # Statsmodels forecast
    row_stats = row[ind]
    row_stats = sm.add_constant(pd.DataFrame([row_stats]), has_constant='add')
    y_hat_stats = model_stats_fcst.predict(row_stats).iloc[0]
    forecast_ols_stats.append(y_hat_stats)
    last_sales_stats = y_hat_stats

    # Sklearn forecast
    row_sklearn = pd.DataFrame([row[ind].values], columns=ind)
    y_hat_sklearn = model_sklearn_fcst.predict(row_sklearn)[0]
    forecast_ols_sklearn.append(y_hat_sklearn)
    last_sales_sklearn = y_hat_sklearn

    # Statsmodel forecast
    forecast_ols_statsmodels.append(y_hat_stats)

# Then after the loop:
X_forward_df['y_fcst_ols_statsmodels'] = forecast_ols_statsmodels


# 8. Assign forecasts to df_combined
df_combined = df.copy()

for i, period in enumerate(future_periods):
    df_combined.loc[period, 'y_fcst_ols_matrix'] = forecast_ols_matrix[i]
    df_combined.loc[period, 'y_fcst_ols_stats'] = forecast_ols_stats[i]
    df_combined.loc[period, 'y_fcst_ols_sklearn'] = forecast_ols_sklearn[i]
    df_combined.loc[period, 'y_fcst_ols_statsmodels'] = X_forward_df.loc[period, 'y_fcst_ols_statsmodels']

    for col in all_vars + [dep_lag]:
        df_combined.loc[period, col] = X_forward_df.loc[period, col]


In [ ]:
# --- Step 7: Fit Bayesian Model and Forecast Forward ---

# 1. Fit Bayesian regression model on full training data
trace = fit_bayesian_regression(X_train_fcst, y_train_fcst)

# 2. Forecast using future inputs (converted and aligned)
simulated_forecasts_bayes = simulate_bayesian_forecasts(X_forward_df[ind].values, trace)
summary_bayes = summarize_bayesian_distribution(simulated_forecasts_bayes)

# 3. Assign forecasts back to df_combined
df_combined.loc[future_periods, 'y_fcst_bayes_mean'] = summary_bayes['mean'].values
df_combined.loc[future_periods, 'y_fcst_bayes_p5'] = summary_bayes['p5'].values
df_combined.loc[future_periods, 'y_fcst_bayes_p95'] = summary_bayes['p95'].values

In [ ]:
# --- Step 8: Residual Bootstrap Forecast ---

# 1. Check if residuals are stationary (optional warning)
check_residual_stationarity(df['residuals'].dropna())

# 2. Pull residuals from training set
residuals_train = df['residuals'].dropna().values

# 3. Run bootstrap simulation using aligned forward inputs
X_future_array_boot = X_forward_df[ind].values
simulated_forecasts_bootstrap = simulate_bootstrap_forecasts(X_future_array_boot, beta_fcst_matrix, residuals_train)

# 4. Summarize forecast distribution
summary_bootstrap = summarize_bootstrap_distribution(simulated_forecasts_bootstrap)

# 5. Add bootstrap results to df_combined
df_combined.loc[future_periods, 'y_fcst_bootstrap'] = summary_bootstrap['mean'].values
df_combined.loc[future_periods, 'y_fcst_bootstrap_p5'] = summary_bootstrap['p5'].values
df_combined.loc[future_periods, 'y_fcst_bootstrap_p95'] = summary_bootstrap['p95'].values

In [ ]:
# --- Step 9: Final Visuals ---
plot_actual_vs_fitted_vs_forecast(df_combined, dep)
plot_selected_forecasts(df_combined, dep)

In [ ]:
# --- Step 10: Summary Tables ---
styled_table = summarize_forecast_table_with_colors(df_combined, future_periods)
styled_table

In [ ]:
df_combined.index.name = 'date'
df_combined.to_csv("grocery_forecast_month.csv")

### FULL CODE ###